In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
html = browser.html
soup_obj = soup(html, 'html.parser')

In [ ]:
browser.execute_script('document.querySelector(".app-btn.hs_btn_login_start").click();')

email_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="email"]'))
)


login = (email) 
email_input.send_keys(login)

In [ ]:
password_input = WebDriverWait(browser.driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '.form-input.medium.clear.input input[type="password"]'))
)
password = (passw) 
password_input.send_keys(password)

In [ ]:
button = browser.find_by_css('.app-btn.hs_btn_login_submit_email').first
button.click()